In [1]:
from static_huff import static_huffman
from adaptive_huff import adaptive_huffman_encode

import static_huff
import adaptive_huff

In [2]:
root = static_huffman("abracadabra")
root

-----HUFFMAN TREE-----
#11
0 -> #5 a
1 -> #6
 0 -> #2
  0 -> #1 c
  1 -> #1 d
 1 -> #4
  0 -> #2 r
  1 -> #2 b

In [3]:
encoded = static_huff.encode("abracadabra")
print(encoded)

bitarray('01111100100010101111100')


In [4]:
decoded = static_huff.decode(encoded, root)
print(decoded)

abracadabra


In [5]:
text = "aa"
adaptive_huff.adaptive_huffman_encode(text, N = 3)

-----HUFFMAN TREE-----
#{ W=0, N=3 }
0 -> #{ W=0, N=1 } NYT
1 -> #{ W=1, N=2 } a
ToUpdate:  -----HUFFMAN TREE-----
#{ W=0, N=3 }
0 -> #{ W=0, N=1 } NYT
1 -> #{ W=1, N=2 } a


KeyError: 2